# Pipeline de Pré Processamento para NLP

# Pré Processamento de texto

Para pipelines de pré processamento baseadadas em compreensão de texto, é crucial que façamos uma normalização e limpeza dos dados antes de analisar-los. Esse pré processamento envolve algumas etapas:

- **Tokenização:** Separar o texto em palavras comuns ou tokens. Por exemplo, "Olá mundo!" irá virar ["ola", "mundo", "!"].
- **Lowercasing:** Converte todos os caracteres em textos minúsculos para garantir uniformidade. Por exemplo, "Ola" e "ola" são tratados igualmente.
- **Remoção de Pontuação:** Eliminar marcas de pontuação já que elas não contribuem com o significado das palavras.
- **Remoção de Stop Words:** Remoção de palavras comuns como "e", "o", "é", que não carregam um significado.
- **Lematização:** Convertendo palavras para a sua forma base. Por exemplo, "correndo" irá virar "correr".
- **Stemming:** Semelhante a lematização, mas geralmente mais agressiva. Ela reduz as palavras para a sua forma raíz, por exemplo "jogando" é reduzido para "jog", que é a raiz.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/IMDB_Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Padronizando os caracteres para lowercase

In [2]:
df['review'][0].lower()

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show is due to the fa

In [3]:
df['review']=df['review'].str.lower()
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


### Removendo Tags HTML utilizando expressões regulares

In [4]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)

In [5]:
df['review']=df['review'].apply(remove_html_tags)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


### Removendo URLs

In [6]:
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

text='Checkout this webpage to learn more about deep learning https://www.deeplearning.ai/ai-notes/initialization/index.html'
remove_url(text)

'Checkout this webpage to learn more about deep learning '

### Removendo Pontuações

In [7]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
exclude=string.punctuation
def remove_punc(text):
    for char in exclude:
        text=text.replace(char,'')
    return text

def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

text='okay, so. this, is/ just a, text, with. punc'
remove_punc(text)

'okay so this is just a text with punc'

### Tratamento de linguagem Geração Z
Para os modelos performarem melhor durante suas interpretações, precisamos que ele compreenda o significado por trás das abreviações feitas na internet, infelizmente significando que temos que ensinar aos modelos um pouco da linguagem da geração Z do Twitter :,(.


In [9]:
chat_word = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': "For What It's Worth",
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'ILU: I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A.. Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A..',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick:-D Laugher',
    'TFW': 'That feeling when',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': "I don't care",
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping, bored, tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'BFF': 'Best friends forever',
    'CSL': "Can't stop laughing"
}

In [10]:
def short_conv(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_word:
            new_text.append(chat_word[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


short_conv("ASAP let me know please")

'As Soon As Possible let me know please'

### Correção Ortográfica

In [11]:
from textblob import TextBlob

text=" ceertainli I dont kniw what is wrrong herre"
textblb=TextBlob(text)
textblb.correct().string

' certainly I dont know what is wrong here'

### Remoção de Stop Words
Stop words são palavras comuns em uma linguagem que muitas vezes são removidas porque carregam pouco peso semântico e podem desordenar a análise. Essas palavras incluem artigos (por exemplo, "a", "the"), conjunções (por exemplo, "e", "ou"), preposições (por exemplo, "in", "on") e outros termos que ocorrem com freqüência que não contribuem significativamente para o significado de uma frase. Ao remover palavras de parada, o foco muda para palavras mais significativas que representam melhor o conteúdo do texto, melhorando a eficiência e a precisão da análise de texto e dos modelos de aprendizado de máquina.

In [12]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords.words("english")

[nltk_data] Downloading package stopwords to C:\Users\Thiago
[nltk_data]     Barros\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
            
    x=new_text[:]
    new_text.clear()
    return " ".join(x)

text="I wasn't sure that this might happened"
remove_stopwords(text)

'I  sure   might happened'

### Tratamento de Emojis
Aqui temos duas formas de tratar os emojis, sendo removendo-os completamente ou apenas substituindo eles pelo o seu valor semântico.

In [14]:
# Removendo os emojis
import re
def remove_emoji(text):
    emoji_pattern=re.compile("["
                             u"\U0001F600-\U0001F64F" #emoticons
                             u"\U0001F300-\U0001F5FF" #symbols, pictograph
                              u"\U0001F680-\U0001F6FF" #transport and map symbol
                              u"\U0001F1E0-\U0001F1FF" # flags(IOS)
                              u"\U00002702-\U000027B0"
                              u"\U00002FC2-\U0001F251"
                             "]+",flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

text="that is not so funny please stop 😭"
remove_emoji(text)

'that is not so funny please stop '

In [15]:
# Substituindo
import emoji
print(emoji.demojize(text))

that is not so funny please stop :loudly_crying_face:


### Tokenização
A tokenização é o passo mais fundamental na NLP que envolve dividir o texto em unidades menores chamadas tokens, que podem ser palavras, frases ou frases. Esse processo ajuda a converter o texto bruto em um formato estruturado para análise. Existem diferentes tipos de tokenização, incluindo tokenização de palavras (dividir texto em palavras individuais) e tokenização de frases (dividir texto em frases). A tokenização adequada considera a pontuação, os espaços e as regras específicas do idioma para representar com precisão a estrutura e o significado do texto, permitindo um processamento de texto e extração de recursos mais eficazes nas tarefas de NLP.

Veremos quatro formas de fazer essa tokenização: 
- Via Python split
- Utilizando expressões regulares
- Usando a biblioteca NLTK
- Usando a biblioteca Spacy

In [16]:
# Usando a função de split
sent1="I am going to delhi"
sent1.split()

['I', 'am', 'going', 'to', 'delhi']

In [17]:
# Usando a função de split
sent1="I am, going to delhi"
sent1.split(',')

['I am', ' going to delhi']

In [18]:
# O problema que surge, ele não consegue separar os '!!!!' 
sent1="I am, going to delhi!!!"
sent1.split(',')

['I am', ' going to delhi!!!']

In [19]:
# Utilizando expressões regulares
import re
tokens=re.findall("[\w']+",sent1)
tokens

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Thiago Barros\AppData\Local\Temp\ipykernel_3472\4056234187.py:3: SyntaxWarning: invalid escape sequence '\w'
  tokens=re.findall("[\w']+",sent1)


['I', 'am', 'going', 'to', 'delhi']

In [20]:
# Utilizando a biblioteca NLTK
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')
word_tokenize(sent1)

[nltk_data] Downloading package punkt_tab to C:\Users\Thiago
[nltk_data]     Barros\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['I', 'am', ',', 'going', 'to', 'delhi', '!', '!', '!']

In [21]:
sent2='I have a Ph.D in M.L'
sent3="We're here to help! mail us at xuz@gmail.com"

In [22]:
word_tokenize(sent2)

['I', 'have', 'a', 'Ph.D', 'in', 'M.L']

In [23]:
# falha nessa sentença, pois separou o id do e-mail no básico de @
word_tokenize(sent3)

['We',
 "'re",
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'xuz',
 '@',
 'gmail.com']

In [26]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ----------- ---------------------------- 3.7/12.8 MB 27.3 MB/s eta 0:00:01
     ---------------------- ----------------- 7.3/12.8 MB 21.6 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 23.9 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 23.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [27]:
# Utilizando o Spacy
import spacy

nlp=spacy.load('en_core_web_sm')

doc1=nlp(sent1)
doc2=nlp(sent2)
doc3=nlp(sent3)

for token in doc2:
    print(token)

I
have
a
Ph
.
D
in
M.L


### Stemming
Stemming reduz as palavras à sua raiz ou forma de base, removendo sufixos. Ao contrário da lematização, que se baseia em regras linguísticas, o stemming usa métodos heurísticos para remover afixos. Por exemplo, "running", "runner," e "ran" podem ser reduzidos a "run." Algoritmos comuns incluem o Porter Stemmer, que aplica uma série de regras a sufixos iterativos, e o Snowball Stemmer, que é uma versão melhorada do Porter Stemmer. Embora o stemming seja mais rápido e mais simples do que a lematização, ele pode ser menos preciso, às vezes produzindo hastes que não são palavras reais.

In [28]:
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

sample="Walk walking walked walks"
stem_words(sample)

'walk walk walk walk'

In [29]:
sample2='Stemming is a text preprocessing technique in NLP that reduces words to their root or base form by removing suffixes. Unlike lemmatization, which relies on linguistic rules, stemming uses heuristic methods to strip affixes. For example, "running," "runner," and "ran" might all be reduced to "run." Common algorithms include the Porter Stemmer, which applies a series of rules to iteratively strip suffixes, and the Snowball Stemmer, which is an improved version of the Porter Stemmer. While stemming is faster and simpler than lemmatization, it can be less accurate, sometimes producing stems that are not actual words.'
stem_words(sample2)

'stem is a text preprocess techniqu in nlp that reduc word to their root or base form by remov suffixes. unlik lemmatization, which reli on linguist rules, stem use heurist method to strip affixes. for example, "running," "runner," and "ran" might all be reduc to "run." common algorithm includ the porter stemmer, which appli a seri of rule to iter strip suffixes, and the snowbal stemmer, which is an improv version of the porter stemmer. while stem is faster and simpler than lemmatization, it can be less accurate, sometim produc stem that are not actual words.'

### Lematização

A lematização reduz as palavras à sua forma base ou raiz, conhecida como lema. Ao contrário da derivação, que simplesmente corta as terminações das palavras, a lematização considera o contexto e a análise morfológica das palavras, garantindo que as palavras sejam transformadas em formas base significativas. Por exemplo, "running" se torna "run" e "better" se torna "good". Esse processo ajuda a padronizar palavras, melhorando a precisão da análise de texto ao agrupar diferentes formas flexionadas de uma palavra em um único item. A lematização é essencial para tarefas como classificação de texto, análise de sentimento e recuperação de informações, pois melhora a qualidade dos recursos extraídos do texto

In [30]:
import spacy

nlp = spacy.load('en_core_web_sm')

sentence = "The children were playing in the park, running and laughing as they enjoyed their freedom, unaware of the time passing quickly by."
doc = nlp(sentence)

print("Original Word - Lemmatized Word")
for token in doc:
    print(f"{token.text} - {token.lemma_}")

Original Word - Lemmatized Word
The - the
children - child
were - be
playing - play
in - in
the - the
park - park
, - ,
running - run
and - and
laughing - laugh
as - as
they - they
enjoyed - enjoy
their - their
freedom - freedom
, - ,
unaware - unaware
of - of
the - the
time - time
passing - pass
quickly - quickly
by - by
. - .


### One hot Encoding
One Hot Encoding é uma técnica usada para converter dados categóricos em um formato numérico que pode ser usado por algoritmos de aprendizado de máquina. Este método transforma cada valor de categoria em uma nova coluna binária e atribui 1 ou 0 (Verdadeiro/Falso) para indicar a presença daquela categoria nos dados.

In [32]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

data = {
    'ID': [1, 2, 3, 4, 5],
    'Color': ['Red', 'Green', 'Blue', 'Green', 'Blue']
}

df = pd.DataFrame(data)

encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['Color']])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['Color']))
final_df = pd.concat([df, encoded_df], axis=1)

print("Original DataFrame:")
print(df)
print("\nOne-Hot Encoded DataFrame:")
print(final_df)

Original DataFrame:
   ID  Color
0   1    Red
1   2  Green
2   3   Blue
3   4  Green
4   5   Blue

One-Hot Encoded DataFrame:
   ID  Color  Color_Blue  Color_Green  Color_Red
0   1    Red         0.0          0.0        1.0
1   2  Green         0.0          1.0        0.0
2   3   Blue         1.0          0.0        0.0
3   4  Green         0.0          1.0        0.0
4   5   Blue         1.0          0.0        0.0


### Bag of Words
A intuição central por trás do modelo Bag of Words (BoW) é que se nosso algoritmo detectar palavras semelhantes ocorrendo com frequências semelhantes, ele classificará os documentos como estando na mesma classe. Aqui, a ordem das palavras e o contexto não importam; o que importa é a presença e a frequência das palavras.

No BoW, cada palavra é representada como uma coordenada em um espaço n-dimensional. Se definirmos o parâmetro binário como True, qualquer palavra que apareça mais de uma vez ainda será tratada como se aparecesse apenas uma vez. Consequentemente, as coordenadas serão binárias, preenchidas apenas com 0s e 1s.

#### Filtrando palavras raras
Para remover palavras raras, você pode usar o parâmetro max_features, que limita o vocabulário às palavras mais frequentes. Por exemplo, definir max_features=1/2 ignora todas as palavras com frequências menores que esse limite.

#### Vantagens do Bag of Words
Simplicidade e Intuitividade: Fácil de implementar e entender.
Mitigação de Problemas de Fora do Vocabulário (OOV): Como o modelo usa um vocabulário fixo, ele evita o problema de OOV que afeta a codificação one-hot.

#### Desvantagens do Bag of Words

- **Esparsidade:** Com um vocabulário grande, a representação terá muitos 0s, potencialmente levando a overfitting.
Perda de Informação: Ignora a ordem das palavras e o contexto, levando à perda de significado semântico.
Exemplo: "Este é um filme muito bom" e "Este não é um filme muito bom" podem ser representados de forma semelhante, o que pode causar problemas com similaridade de cosseno.

#### Similaridade de cosseno
A similaridade de cosseno mede o cosseno do ângulo entre dois vetores, fornecendo uma métrica para sua similaridade. A fórmula é:

#TODO: colocar a fórmula aqui.

Usando similaridade de cosseno, "Este é um filme muito bom" e "Este não é um filme muito bom" podem parecer semelhantes devido ao ângulo próximo entre suas representações vetoriais no modelo BoW.

Ao usar o BoW, você pode converter dados de texto em recursos numéricos para algoritmos de aprendizado de máquina de forma rápida e eficiente. No entanto, esteja ciente de suas limitações e considere técnicas mais avançadas como TF-IDF, incorporações de palavras ou modelos de linguagem para capturar informações textuais mais matizadas.


In [33]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Definindo o dataset
data = {
    'Index': ['D1', 'D2', 'D3', 'D4'],
    'Text': [
        'People Follow Mr.Beast',
        'Mr.Beast Follow Mr.Beast',
        'People Write comment',
        'Mr.Beast Write comment'
    ],
    'Output': [1, 1, 0, 0]
}

# Criando um dataframe
df = pd.DataFrame(data)
df

,Index,Text,Output
0,D1,People Follow Mr.Beast,1
1,D2,Mr.Beast Follow Mr.Beast,1
2,D3,People Write comment,0
3,D4,Mr.Beast Write comment,0


In [34]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Text'])

vocab = vectorizer.get_feature_names_out()
print(vocab)

['beast' 'comment' 'follow' 'mr' 'people' 'write']


In [35]:
print(vectorizer.vocabulary_)

{'people': 4, 'follow': 2, 'mr': 3, 'beast': 0, 'write': 5, 'comment': 1}


In [39]:
bow_df = pd.DataFrame(X.toarray(), columns=vocab)

final_df = pd.concat([df[['Index', 'Output']], bow_df], axis=1)

print("DataFrame Original:")
print(df)
print("\nBag of Words DataFrame com os parâmetros binários como False:")
print(final_df)

DataFrame Original:
  Index                      Text  Output
0    D1    People Follow Mr.Beast       1
1    D2  Mr.Beast Follow Mr.Beast       1
2    D3      People Write comment       0
3    D4    Mr.Beast Write comment       0

Bag of Words DataFrame com os parâmetros binários como False:
  Index  Output  beast  comment  follow  mr  people  write
0    D1       1      1        0       1   1       1      0
1    D2       1      1        0       1   1       0      0
2    D3       0      0        1       0   0       1      1
3    D4       0      1        1       0   1       0      1


In [38]:
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['Text'])

vocab = vectorizer.get_feature_names_out()

bow_df = pd.DataFrame(X.toarray(), columns=vocab)

final_df = pd.concat([df[['Index', 'Output']], bow_df], axis=1)

print("Dataframe original:")
print(df)
print("\nBag of Words Dataframe com os parâmetros binários como True:")
print(final_df)

Dataframe original:
  Index                      Text  Output
0    D1    People Follow Mr.Beast       1
1    D2  Mr.Beast Follow Mr.Beast       1
2    D3      People Write comment       0
3    D4    Mr.Beast Write comment       0

Bag of Words Dataframe com os parâmetros binários como True:
  Index  Output  beast  comment  follow  mr  people  write
0    D1       1      1        0       1   1       1      0
1    D2       1      1        0       1   1       0      0
2    D3       0      0        1       0   0       1      1
3    D4       0      1        1       0   1       0      1


# Para fazer
## Testar mais métodos de pré-processamento:
- N-Grams
- TD-IDF
- WordNet
- Word2Vec
- RoPE (Robust Positional Embeddings)

## Automatizar a escolha e análise do modelo
- AutoSKlearn
- TPOT
- SMAC (sequential model-based algorithm configuration)

## Pegar os melhores modelos e fazer a escolha dos hiperparâmetros
- Utilizar métodos de GridSearch e XGBoost

## Salvar o melhor modelo dentro de um padrão de versionamento
- Salvar o modelo compilado para que seja utilizado em uma API